In [1]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [4]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [5]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [6]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [7]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [8]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

In [9]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [10]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..
The document introduces MetaGPT, a meta-programming framework for LLM-based multi-agent collaboration that enhances problem-solving processes by incorporating human-like Standardized Operating Procedures (SOPs). MetaGPT assigns specific roles to agents, streamlines workflows, and improves task decomposition. It utilizes structured outputs and a communication protocol to ensure efficient collaboration among agents with diverse expertise. The framework achieves state-of-the-art performance in code generation benchmarks and offers a robust and efficient solution for developing LLM-based multi-agent systems. Additionally, the document discusses the structured workflow in software development within MetaGPT, emphasizing the roles of different agents like Product Manager, Architect, Engineer, and QA Engineer. It highlights the importance of role specialization, workflo

In [11]:
print(len(response.source_nodes))

34


In [12]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context from the MetaGPT paper, which would likely include information on how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they can publish structured messages. This shared message pool allows all agents to exchange messages directly, enabling them to access messages from other entities transparently. Additionally, agents can subscribe to relevant messages based on their role profiles, allowing them to extract the information they need for their specific tasks and responsibilities.


In [13]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [14]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: The ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results show that MetaGPT effectively addresses challenges related to context utilization, code hallucinations, and information overload in software development. By accurately unfolding natural language descriptions and maintaining information validity, MetaGPT eliminates ambiguity and enables Language Models (LLMs) to focus on relevant data. Additionally, by focusing on granular tasks like requirement analysis and package selection, MetaGPT guides LLMs in software generation, reducing code hallucination issues. To tackle information overload, MetaGPT utilizes a global message pool and a subscription mechanism to streamline communication and filter out irrelevant contexts, enhancing the efficiency and relevance of information.
